In [1]:
import librosa
import numpy as np
from scipy.fft import fft, fftfreq, rfft, rfftfreq
import pandas as pd

In [2]:
AUDIO = "D:\\Github\\Music-Rec-RAC-DataSci\\org song\\วงวน - SERIOUS BACON ( ORIGINAL by ONEONE )\\vocals.mp3"

y, sr = librosa.load(AUDIO)
onset_env = librosa.onset.onset_strength(y=y, sr=sr)
onset_frames = librosa.onset.onset_detect(onset_envelope=onset_env)
timestamps = librosa.frames_to_time(onset_frames, sr=sr)
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

print("onset_env", onset_env)
print("onset_frames",onset_frames)
print("timestamps",timestamps)

onset_env [0. 0. 0. ... 0. 0. 0.]
onset_frames [  233   253   268   283   381   393   400   415   436   454   485   509
   517   545   692   704   723   735   750   767   849   865   906   922
   928   955   959   983   989  1013  1018  1145  1157  1163  1172  1194
  1205  1221  1236  1394  1398  1409  1412  1429  1440  1443  1456  1465
  1471  1489  1507  1640  1646  1663  1676  1692  1702  1707  1728  1751
  1789  1793  1799  1808  1824  1845  1865  1868  1929  2098  2131  2143
  2147  2162  2177  2203  2218  2236  2279  2338  2367  2379  2382  2397
  2454  2457  2498  2513  2542  2574  2584  2603  2629  2632  2679  2689
  2704  2720  2732  2749  2766  2795  2805  2808  2835  2848  2851  2877
  2906  2925  2940  2980  3027  3041  3043  3056  3072  3084  3099  3112
  3145  3157  3161  3190  3219  3292  3306  3318  3335  3394  3412  3423
  3436  3453  3481  3502  3512  3525  3540  3543  3569  3619  3629  3645
  3656  3684  3690  3704  3732  3737  3748  3762  3775  3791  3804  3820
  38

In [3]:
data_dic = []

for i, onset_frame in enumerate(onset_frames):
    start_time = librosa.frames_to_time(onset_frame, sr=sr)
    end_time = librosa.frames_to_time(onset_frames[i+1] if i+1 < len(timestamps) else onset_frames[-1], sr=sr)
    
    # print("start_time", start_time)
    # print("end_time", end_time)
    
    dataToRead = y[int(start_time * sr) : int(end_time * sr) + 1]

    N = len(dataToRead)
    yf = rfft(dataToRead)
    xf = rfftfreq(N, 1 / sr)
    
    idx = np.argmax(np.abs(yf))
    freq = xf[idx]
    
    data_dic.append({
        "start_time": start_time,
        "end_time": end_time,
        "freq": freq
    })
    
df = pd.DataFrame(data_dic[:-1])
df['note']=df['freq'].map(lambda x: librosa.hz_to_note(x) if x > 0 else None)

<Axes: xlabel='note'>

In [4]:
df['note'].value_counts().plot(kind='bar')

In [5]:
df

,start_time,end_time,freq,note
0,5.410249,5.874649,329.458008,E4
1,5.874649,6.222948,309.997397,D♯4
2,6.222948,6.571247,244.011196,B3
3,6.571247,8.846803,277.289396,C♯4
4,8.846803,9.125442,279.886086,C♯4
...,...,...,...,...
450,272.741587,272.811247,832.075472,G♯5
451,272.811247,273.159546,889.923187,A5
452,273.159546,273.554286,823.233774,G♯5
453,273.554286,273.809705,743.742233,F♯5
